<a href="https://colab.research.google.com/github/jj-snuIS/meta_sql/blob/main/%5BSQL_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_5%EC%A3%BC%EC%B0%A8%5D_Plotly%EC%8B%9C%EA%B0%81%ED%99%94_%26_%EB%A7%88%EC%BC%80%ED%8C%85_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_%EA%B3%B5%EC%9C%A0%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 라이브러리 import

In [ ]:
import pandas as pd

# BigQuery 연동을 위한 기본 설정

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'crested-drive-372522' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

# BigQuery 데이터 불러오기

In [ ]:
query_job = client.query("""
    select *
    from `crested-drive-372522.my_temp.daily_login_users`
""")

loginUsersDf = query_job.to_dataframe()
loginUsersDf['login_date'] = loginUsersDf['login_date'].astype('datetime64')
#------------------------------------------------------------------------------------------------------------------------------------------------#
query_job = client.query("""
    select *
    from `crested-drive-372522.my_temp.daily_orders`
""")

ordersDf = query_job.to_dataframe()
ordersDf['order_date'] = ordersDf['order_date'].astype('datetime64')

#------------------------------------------------------------------------------------------------------------------------------------------------#
query_job = client.query("""
    select pickup_shop_name, order_year_month, order_date
         , sum(payment_amount) as sales
         , count(distinct user_id) as	pu
         , count(distinct order_id) as purchase_count	
         , round(sum(payment_amount)/count(distinct user_id), 2) as arppu
    from `crested-drive-372522.my_temp.daily_orders`
    group by pickup_shop_name, order_year_month, order_date
""")

salesIndexDf = query_job.to_dataframe()
salesIndexDf['order_date'] = salesIndexDf['order_date'].astype('datetime64')

#------------------------------------------------------------------------------------------------------------------------------------------------#
query_job = client.query("""
    select pickup_shop_name, sum(payment_amount) as sales, count(distinct user_id) as pu
    from `crested-drive-372522.localparm_dw.orders`
    group by pickup_shop_name
""")

salesDf = query_job.to_dataframe()

#------------------------------------------------------------------------------------------------------------------------------------------------#
query_job = client.query("""
    select *
    from `crested-drive-372522.localparm_dw.pickup_shop`
""")
shopsDf = query_job.to_dataframe()

# Plotly를 활용한 시각화

## plotly.express

### 막대그래프

In [ ]:
import plotly.express as px

fig = px.bar(salesDf, 
             x ='pickup_shop_name', 
             y = 'sales', 
             color='pickup_shop_name')

# 참고 : https://plotly.com/python/categorical-axes/
# << category를 기준으로 오름차순 정렬 >>
fig.update_xaxes(categoryorder='category ascending')

# << total value를 기준으로 내림차순 정렬 >>
# fig.update_xaxes(categoryorder='total descending')

# << 지정된 category 순서에 맞춰 정렬 >>
# fig.update_xaxes(categoryorder='array', categoryarray= ['픽업존(다이소 난곡사거리점)','픽업존(유승상가)','픽업존(참맛부대찌개아구찜)','픽업존(GS25 목동3동점)','픽업존(더현대서울)','픽업존(GS영등포당산점)','픽업존(서울드림신용협동조합)','픽업존(메이비카페)'])

fig.show(renderer='colab')

### 누적 막대그래프

In [ ]:
import plotly.express as px

## 데이터 핸들링
salesTemp = ordersDf.groupby(['order_year_month', 'pickup_shop_name'])['payment_amount'].sum().reset_index()

## 시각화
fig = px.bar(salesTemp, 
              x="order_year_month", 
              y="payment_amount", 
              color="pickup_shop_name", 
              text="payment_amount")

# 참고: https://plotly.com/python/text-and-annotations/
# << 값을 표시방식을 설정 >>
# fig.update_traces(texttemplate='%{text:,} 원',)# textposition='inside',) 

fig.show(renderer='colab') 

### 라인 그래프

In [ ]:
## << 샵별 일간 매출 추이 - 라인 그래프 >>

## 데이터 핸들링
salesTemp = ordersDf.groupby(['order_date', 'pickup_shop_name'])['payment_amount'].sum().reset_index()

## 시각화
import plotly.express as px

fig = px.line(salesTemp, 
              x="order_date", 
              y="payment_amount", 
              color="pickup_shop_name", 
              # markers=True,
             )

fig.show(renderer='colab')

### 산점도

In [ ]:
## << PU & SALES 산점도 >>

## 데이터 핸들링
salesTemp = ordersDf.groupby(['pickup_shop_name', 'order_date']).agg({'payment_amount':'sum', 'user_id':'nunique'}).reset_index()
salesTemp.rename(columns={'payment_amount':'sales', 'user_id':'pu'}, inplace=True)
salesTemp['sales'] = salesTemp['sales'].astype('float') ## catter plot error 방지

import plotly.express as px

## 시각화
fig = px.scatter(salesTemp, x='pu', y='sales')#, color='pickup_shop_name', size = 'sales', trendline = 'ols')
fig.show()


In [ ]:
## << PU & SALES 산점도 - 픽업 매장별로 분리해서 도식 >>

## 시각화
import plotly.express as px

fig = px.scatter(salesTemp, x='pu', y='sales', 
                 facet_col='pickup_shop_name',) # 표를 분리하기 위한 기준 컬럼 설정 (행기준 적용시 > facet_row 사용)
                 # color='sales', trendline = 'ols',) 
fig.show()

In [ ]:
## << PU & SALES 산점도 - 픽업 매장별 분리 도식 >>

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = salesTemp[salesTemp['pickup_shop_name'] == '픽업존(유승상가)']['pu'],
        y = salesTemp[salesTemp['pickup_shop_name'] == '픽업존(유승상가)']['sales'],
        name = "픽업존(유승상가)",
        mode = "markers", # lines+markers
        # marker_color = "lightgray",
    )
)

fig.add_trace(
    go.Scatter(
        x = salesTemp[salesTemp['pickup_shop_name'] == '픽업존(GS영등포당산점)']['pu'],
        y = salesTemp[salesTemp['pickup_shop_name'] == '픽업존(GS영등포당산점)']['sales'],
        name = "픽업존(GS영등포당산점)",
        mode = "markers", # lines+markers
        # marker_color = "crimson",
    )
)

# fig의 mode에 대해서는 밑에서 일괄 적용하는것이 가능하다.
# fig.update_traces(mode='markers', marker_line_width=0.5, marker_size=10) 

fig.show()

In [ ]:
salesIndexDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 897 entries, 0 to 896
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   pickup_shop_name  897 non-null    object        
 1   order_year_month  897 non-null    object        
 2   order_date        897 non-null    datetime64[ns]
 3   sales             897 non-null    Int64         
 4   pu                897 non-null    Int64         
 5   purchase_count    897 non-null    Int64         
 6   arppu             897 non-null    float64       
dtypes: Int64(3), datetime64[ns](1), float64(1), object(2)
memory usage: 51.8+ KB


In [ ]:
import plotly.express as px

# 데이터 핸들링

# trendline 설정 error 수정.
salesIndexDf['pu'] = salesIndexDf['pu'].astype('float')
salesIndexDf['sales'] = salesIndexDf['sales'].astype('float')

fig = px.scatter(salesIndexDf,
                 x = 'pu',
                 y = 'sales',
                 color = 'pickup_shop_name',
                 facet_col = 'order_year_month', )# 표를 분리하기 위한 기준 컬럼 설정 (행기준 적용시 > facet_row 사용)
                #  facet_col_wrap=4, # 몇개를 기준으로 줄바꿈을 할지 설정 
                #  trendline='ols')

fig.show()

In [ ]:
## << 설정한 컬럼들간의 산점도 >>

import plotly.express as px

fig = px.scatter_matrix(salesIndexDf, 
                        dimensions = ['sales', 'arppu', 'purchase_count', 'pu'],
                        # color = 'pickup_shop_name' # 매장별로 차이가 존재할까?
                        )

fig.show()

# 마케팅 데이터 분석

## 비용 (Cost)

### 일간 비용

In [ ]:
## << 데이터프레임 생성 - 일간 마케팅 비용 >>
query_job = client.query("""
  select aggregate_date, min(format_date('%Y-%m', aggregate_date)) as aggregate_year_month, sum(cost) as cost
  from `my_temp.daily_cost`
  group by aggregate_date
  order by aggregate_date
""")

dailyCostDf = query_job.to_dataframe()
dailyCostDf['aggregate_date'] = dailyCostDf['aggregate_date'].astype('datetime64')

In [ ]:
## << 시각화 - 일간 마케팅 비용 막대그래프 >>

import plotly.express as px

fig = px.bar(dailyCostDf, 
             x ='aggregate_date', 
             y = 'cost', 
             color='aggregate_year_month',)

fig.show(renderer='colab')

In [ ]:
dailyCostDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   aggregate_date        214 non-null    datetime64[ns]
 1   aggregate_year_month  214 non-null    object        
 2   cost                  214 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 5.1+ KB


### 월간 비용 

In [ ]:
## << 시각화 - 월간 마케팅 비용 >>

import plotly.express as px

fig = px.bar(dailyCostDf, 
             x ='aggregate_year_month', 
             y = 'cost', 
             color='aggregate_year_month',)

fig.show(renderer='colab')

In [ ]:
## << 데이터프레임 생성 - 월간 마케팅 비용 >>

query_job = client.query("""
  select format_date('%Y-%m', aggregate_date) as aggregate_year_month, round(sum(cost)) as cost
  from `my_temp.daily_cost`
  group by aggregate_year_month
  order by aggregate_year_month
""")

monthlyCostDf = query_job.to_dataframe()

In [ ]:
## << 시각화 - 월간 마케팅 비용 >>

import plotly.express as px

fig = px.bar(monthlyCostDf, 
             x ='aggregate_year_month', 
             y = 'cost', 
             color='aggregate_year_month',)

fig.show(renderer='colab')

## 모객 유저수

### 일간 모객 유저수

In [ ]:
## << 데이터프레임 생성 - 일간 가입 유저수 >>

query_job = client.query("""
  with userinfo as (
    select b.attribute_type, b.campaign
        , b.install_time, a.create_date, a.create_year_month
        , a.attribute_id, a.user_id, a.nickname, a.account_type, a.platform
        , a.region_1depth_name, a.region_2depth_name, a.region_3depth_name
        , a.h_code, a.x, a.y
    from `my_temp.marketing_myinfos` as a
    left join (
      select 'organic' as attribute_type, install_time, attribute_id, campaign
      from `my_temp.organic_install`

      union all 

      select 'non_organic' as attribute_type, install_time, attribute_id, campaign
      from `my_temp.non_organic_install`
    ) as b
    on a.attribute_id = b.attribute_id and a.create_date = b.install_time
  )

  select create_date, attribute_type, count(distinct user_id) as user_cnt
  from userinfo
  group by create_date, attribute_type 
  order by create_date, attribute_type 
""")

dailyJoinDf = query_job.to_dataframe()

In [ ]:
## << 시각화 - Attribute 타입별 일간 가입 유저수  >>

import plotly.express as px

fig = px.bar(dailyJoinDf, 
             x ='create_date', 
             y = 'user_cnt', 
             color='attribute_type',)

fig.show(renderer='colab')

### 월간 모객 유저수

In [ ]:
## << 데이터 프레임 생성 - 월간 가입 유저수 >>

query_job = client.query("""
  with userinfo as (
    select b.attribute_type, b.campaign
        , b.install_time, a.create_date, a.create_year_month
        , a.attribute_id, a.user_id, a.nickname, a.account_type, a.platform
        , a.region_1depth_name, a.region_2depth_name, a.region_3depth_name
        , a.h_code, a.x, a.y
    from `my_temp.marketing_myinfos` as a
    left join (
      select 'organic' as attribute_type, install_time, attribute_id, campaign
      from `my_temp.organic_install`

      union all 

      select 'non_organic' as attribute_type, install_time, attribute_id, campaign
      from `my_temp.non_organic_install`
    ) as b
    on a.attribute_id = b.attribute_id and a.create_date = b.install_time
  )

  select create_year_month, attribute_type, count(distinct user_id) as user_cnt
  from userinfo
  group by create_year_month, attribute_type 
  order by create_year_month, attribute_type 
""")

MonthlyJoinDf = query_job.to_dataframe()

In [ ]:
## << 시각화 - Attribute 타입별 월간 가입 유저수  >>

import plotly.express as px

fig = px.bar(MonthlyJoinDf, 
             x ='create_year_month', 
             y = 'user_cnt', 
             color= 'attribute_type',)

fig.show(renderer='colab')

## LTV (Life Time Value)

### 월간 모객유저 LTV

In [ ]:
## << 데이터프레임 생성 - 유저그룹(all/organic/non_organic), 월간 가입유저수, SALES, LTV  >> - 추적기간이 차지 않은 그룹까지도 sales, ltv가 집계됨(코드상에서 처리할 수는 없을까요?)

query_job = client.query("""
  with userinfo as (
    select b.attribute_type, b.campaign
        , b.install_time, a.create_date, a.create_year_month
        , a.attribute_id, a.user_id, a.nickname, a.account_type, a.platform
        , a.region_1depth_name, a.region_2depth_name, a.region_3depth_name
        , a.h_code, a.x, a.y
    from `my_temp.marketing_myinfos` as a
    left join (
      select 'organic' as attribute_type, install_time, attribute_id, campaign
      from `my_temp.organic_install`

      union all 

      select 'non_organic' as attribute_type, install_time, attribute_id, campaign
      from `my_temp.non_organic_install`
    ) as b
    on a.attribute_id = b.attribute_id and a.create_date = b.install_time
  ),

  daily_orders as (
    select order_date, user_id, sum(payment_amount) as payment_amount
    from `my_temp.daily_orders`
    where order_date >= '2022-05-01'
    group by order_date, user_id
  ),

  ltv_set as (
    select 'D15' as ltv_period 
        , a.create_year_month, a.attribute_type
        , count(distinct a.user_id) as user_cnt
        , sum(d15.payment_amount) as sales
        , round(sum(d15.payment_amount)/count(distinct a.user_id)) as ltv
    from userinfo as a
    left join daily_orders as d15
    on a.user_id = d15.user_id and a.create_date <= d15.order_date and date_add(a.create_date, interval 14 day) >= d15.order_date 
    group by rollup(a.create_year_month, a.attribute_type)

    union all

    select 'D30' as ltv_period 
        , a.create_year_month, a.attribute_type
        , count(distinct a.user_id) as user_cnt
        , sum(d30.payment_amount) as sales
        , round(sum(D30.payment_amount)/count(distinct a.user_id)) as ltv
    from userinfo as a
    left join daily_orders as d30
    on a.user_id = d30.user_id and a.create_date <= d30.order_date and date_add(a.create_date, interval 29 day) >= d30.order_date 
    group by rollup(a.create_year_month, a.attribute_type)

    union all

    select 'D45' as ltv_period 
        , a.create_year_month, a.attribute_type
        , count(distinct a.user_id) as user_cnt
        , sum(d45.payment_amount) as sales
        , round(sum(D45.payment_amount)/count(distinct a.user_id)) as ltv
    from userinfo as a
    left join daily_orders as d45
    on a.user_id = d45.user_id and a.create_date <= d45.order_date and date_add(a.create_date, interval 44 day) >= d45.order_date 
    group by rollup(a.create_year_month, a.attribute_type)

    union all

    select 'D60' as ltv_period 
        , a.create_year_month, a.attribute_type
        , count(distinct a.user_id) as user_cnt
        , sum(d60.payment_amount) as sales
        , round(sum(D60.payment_amount)/count(distinct a.user_id)) as ltv
    from userinfo as a
    left join daily_orders as d60
    on a.user_id = d60.user_id and a.create_date <= d60.order_date and date_add(a.create_date, interval 59 day) >= d60.order_date 
    group by rollup(a.create_year_month, a.attribute_type)

    union all

    select 'D75' as ltv_period 
        , a.create_year_month, a.attribute_type
        , count(distinct a.user_id) as user_cnt
        , sum(d75.payment_amount) as sales
        , round(sum(D75.payment_amount)/count(distinct a.user_id)) as ltv
    from userinfo as a
    left join daily_orders as d75
    on a.user_id = d75.user_id and a.create_date <= d75.order_date and date_add(a.create_date, interval 74 day) >= d75.order_date 
    group by rollup(a.create_year_month, a.attribute_type)

    union all

    select 'D90' as ltv_period 
        , a.create_year_month, a.attribute_type
        , count(distinct a.user_id) as user_cnt
        , sum(d90.payment_amount) as sales   
        , round(sum(D90.payment_amount)/count(distinct a.user_id)) as ltv
    from userinfo as a
    left join daily_orders as d90
    on a.user_id = d90.user_id and a.create_date <= d90.order_date and date_add(a.create_date, interval 89 day) >= d90.order_date 
    group by rollup(a.create_year_month, a.attribute_type)
  )

  select ltv_period, create_year_month
      , case when attribute_type is null then 'all' else attribute_type end as attribute_type
      , user_cnt, sales, ltv 
  from ltv_set
  where create_year_month is not null
  order by create_year_month, attribute_type, ltv_period 
""")

ltvDf = query_job.to_dataframe()


In [ ]:
## << 데이터프레임 생성 - 유저그룹(all/organic/non_organic), 월간 가입유저수, SALES, LTV  >> - 추적기간이 차지 않은 그룹은 sales, ltv에 null값 할당

query_job = client.query("""
  with userinfo as (
    select b.attribute_type, b.campaign
        , b.install_time, a.create_date, a.create_year_month
        , a.attribute_id, a.user_id, a.nickname, a.account_type, a.platform
        , a.region_1depth_name, a.region_2depth_name, a.region_3depth_name
        , a.h_code, a.x, a.y
    from `my_temp.marketing_myinfos` as a
    left join (
      select 'organic' as attribute_type, install_time, attribute_id, campaign
      from `my_temp.organic_install`

      union all 

      select 'non_organic' as attribute_type, install_time, attribute_id, campaign
      from `my_temp.non_organic_install`
    ) as b
    on a.attribute_id = b.attribute_id and a.create_date = b.install_time
  ),

  daily_orders as (
    select order_date, user_id, sum(payment_amount) as payment_amount
    from `my_temp.daily_orders`
    where order_date >= '2022-05-01'
    group by order_date, user_id
  ),

  ltv_set as (
    select 'D15' as ltv_period, a.user_id   
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 14 then 0 else d15.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d15
    on a.user_id = d15.user_id and a.create_date <= d15.order_date and date_add(a.create_date, interval 14 day) >= d15.order_date 

    union all

    select 'D30' as ltv_period, a.user_id   
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 29 then 0 else d30.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d30
    on a.user_id = d30.user_id and a.create_date <= d30.order_date and date_add(a.create_date, interval 29 day) >= d30.order_date 

    union all

    select 'D45' as ltv_period, a.user_id  
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 44 then 0 else d45.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d45
    on a.user_id = d45.user_id and a.create_date <= d45.order_date and date_add(a.create_date, interval 44 day) >= d45.order_date 

    union all

    select 'D60' as ltv_period, a.user_id  
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 59 then 0 else d60.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d60
    on a.user_id = d60.user_id and a.create_date <= d60.order_date and date_add(a.create_date, interval 59 day) >= d60.order_date 

    union all

    select 'D75' as ltv_period, a.user_id 
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 74 then 0 else d75.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d75
    on a.user_id = d75.user_id and a.create_date <= d75.order_date and date_add(a.create_date, interval 74 day) >= d75.order_date 

    union all

    select 'D90' as ltv_period, a.user_id
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 89 then 0 else d90.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d90
    on a.user_id = d90.user_id and a.create_date <= d90.order_date and date_add(a.create_date, interval 89 day) >= d90.order_date 

  ),

  agg_ltv_group as (
    select ltv_period, create_year_month, attribute_type
        , count(distinct user_id) as user_cnt
        , sum(payment_amount) as sales
        , round(sum(payment_amount)/count(distinct user_id)) as ltv 
        , count(distinct case when payment_amount = 0 then user_id end) as check_cnt
    from ltv_set
    group by rollup(ltv_period, create_year_month, attribute_type)
  )

  select ltv_period, create_year_month
      , case when attribute_type is null then 'all' else attribute_type end as attribute_type
      , user_cnt
      , case when check_cnt = 0 then sales else null end as sales
      , case when check_cnt = 0 then ltv else null end as ltv
  from agg_ltv_group
  where create_year_month is not null
  order by ltv_period, create_year_month, attribute_type
""")

ltvDf = query_job.to_dataframe()

In [ ]:
## < 시각화 - 월간 가입 유저 그룹별 LTV >
allLtvDf = ltvDf[ltvDf['attribute_type'] == 'all']
organicLtvDf = ltvDf[ltvDf['attribute_type'] == 'organic']
nonOrganicvDf = ltvDf[ltvDf['attribute_type'] == 'non_organic']

# 참고 : https://community.plotly.com/t/can-plotly-support-2-x-axis-and-2-y-axis-in-one-graph/38303/16

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=1)

fig.add_trace(go.Bar(x = [allLtvDf['create_year_month'].tolist(), allLtvDf['ltv_period'].tolist()], y= allLtvDf['ltv'].tolist(), name = "all"), row=1, col=1)  #name=“yaxis1 data”, yaxis=“y2”), row=1, col=2)
fig.add_trace(go.Bar(x = [organicLtvDf['create_year_month'].tolist(), organicLtvDf['ltv_period'].tolist()], y= organicLtvDf['ltv'].tolist(), name = "organic"), row=2, col=1)  #name=“yaxis1 data”, yaxis=“y2”), row=1, col=2)
fig.add_trace(go.Bar(x = [nonOrganicvDf['create_year_month'].tolist(), nonOrganicvDf['ltv_period'].tolist()], y= nonOrganicvDf['ltv'].tolist(), name = "non-organic"), row=3, col=1)  #name=“yaxis1 data”, yaxis=“y2”), row=1, col=2)
fig.show(renderer='colab')

## ROAS (Return on Ad Spend)

### 월간 모객유저 ROAS

In [ ]:
## << 데이터프레임 생성 - Attribute 타입별 월간 가입 유저 ROAS  >>

query_job = client.query("""
  with userinfo as (
    select b.attribute_type, b.campaign
        , b.install_time, a.create_date, a.create_year_month
        , a.attribute_id, a.user_id, a.nickname, a.account_type, a.platform
        , a.region_1depth_name, a.region_2depth_name, a.region_3depth_name
        , a.h_code, a.x, a.y
    from `my_temp.marketing_myinfos` as a
    left join (
      select 'organic' as attribute_type, install_time, attribute_id, campaign
      from `my_temp.organic_install`

      union all 

      select 'non_organic' as attribute_type, install_time, attribute_id, campaign
      from `my_temp.non_organic_install`
    ) as b
    on a.attribute_id = b.attribute_id and a.create_date = b.install_time
  ),

  daily_orders as (
    select order_date, user_id, sum(payment_amount) as payment_amount
    from `my_temp.daily_orders`
    where order_date >= '2022-05-01'
    group by order_date, user_id
  ),

  ltv_set as (
    select 'D15' as ltv_period, a.user_id   
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 14 then 0 else d15.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d15
    on a.user_id = d15.user_id and a.create_date <= d15.order_date and date_add(a.create_date, interval 14 day) >= d15.order_date 

    union all

    select 'D30' as ltv_period, a.user_id   
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 29 then 0 else d30.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d30
    on a.user_id = d30.user_id and a.create_date <= d30.order_date and date_add(a.create_date, interval 29 day) >= d30.order_date 

    union all

    select 'D45' as ltv_period, a.user_id  
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 44 then 0 else d45.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d45
    on a.user_id = d45.user_id and a.create_date <= d45.order_date and date_add(a.create_date, interval 44 day) >= d45.order_date 

    union all

    select 'D60' as ltv_period, a.user_id  
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 59 then 0 else d60.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d60
    on a.user_id = d60.user_id and a.create_date <= d60.order_date and date_add(a.create_date, interval 59 day) >= d60.order_date 

    union all

    select 'D75' as ltv_period, a.user_id 
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 74 then 0 else d75.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d75
    on a.user_id = d75.user_id and a.create_date <= d75.order_date and date_add(a.create_date, interval 74 day) >= d75.order_date 

    union all

    select 'D90' as ltv_period, a.user_id
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 89 then 0 else d90.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d90
    on a.user_id = d90.user_id and a.create_date <= d90.order_date and date_add(a.create_date, interval 89 day) >= d90.order_date 

  ),

  agg_ltv_group as (
    select ltv_period, create_year_month, attribute_type
        , count(distinct user_id) as user_cnt
        , sum(payment_amount) as sales
        , round(sum(payment_amount)/count(distinct user_id)) as ltv 
        , count(distinct case when payment_amount = 0 then user_id end) as check_cnt
    from ltv_set
    group by rollup(ltv_period, create_year_month, attribute_type)
  ),

  agg_ltv_group_final as (
    select ltv_period, create_year_month
        , case when attribute_type is null then 'all' else attribute_type end as attribute_type
        , user_cnt
        , case when check_cnt = 0 then sales else null end as sales -- else 0 으로 하면, 시각화 할때 어떤 차이가 날까요?
        , case when check_cnt = 0 then ltv else null end as ltv     -- else 0 으로 하면, 시각화 할때 어떤 차이가 날까요?
    from agg_ltv_group
    where create_year_month is not null
      -- and attribute_type = 'non_organic'
    order by ltv_period, create_year_month, attribute_type
  ), 

  monthly_cost as (
    select format_date('%Y-%m', aggregate_date) as aggregate_year_month, sum(cost) as cost
    from `my_temp.daily_cost`
    group by aggregate_year_month
    order by aggregate_year_month
  )

  select a.ltv_period, a.create_year_month, a.attribute_type
      , a.user_cnt, a.sales, a.ltv, b.cost
      , round(a.sales/b.cost, 3) as roas 
  from agg_ltv_group_final as a
  left join monthly_cost as b
  on a.create_year_month = b.aggregate_year_month
  order by create_year_month, attribute_type, ltv_period 
""")

roasDf = query_job.to_dataframe()

In [ ]:
# 11월은 지표 추적기간이 다차지 않았으므로 데이터에서 제외 처리
allRoasDf = roasDf[(roasDf['attribute_type'] == 'all') & (roasDf['create_year_month'] != '2022-11')] 
organicRoasDf = roasDf[(roasDf['attribute_type'] == 'organic') & (roasDf['create_year_month'] != '2022-11')]
nonOrganicRoasDf = roasDf[(roasDf['attribute_type'] == 'non_organic') & (roasDf['create_year_month'] != '2022-11')]

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 그리고자 하는 서브플랏 골격 설정 (3x1로 설정, y축 이중축 허용, 그래프간 간격 설정)
fig = make_subplots(rows=3, cols=1, 
                    specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]], 
                    vertical_spacing = 0.15)

# fig에 go.Bar를 사용해서 LTV 막대그래프 그리기 
fig.add_trace(go.Bar(x = [allRoasDf['create_year_month'].tolist(), allRoasDf['ltv_period'].tolist()], 
                     y= allRoasDf['ltv'].tolist(), name = "LTV(all)"), 
              secondary_y=False, 
              row=1, col=1)  
fig.add_trace(go.Bar(x = [organicRoasDf['create_year_month'].tolist(), organicRoasDf['ltv_period'].tolist()], 
                     y= organicRoasDf['ltv'].tolist(), name = "LTV(organic)"), 
              secondary_y=False, 
              row=2, col=1) 
fig.add_trace(go.Bar(x = [nonOrganicRoasDf['create_year_month'].tolist(), nonOrganicRoasDf['ltv_period'].tolist()], 
                     y= nonOrganicRoasDf['ltv'].tolist(), name = "LTV(non-organic)"), 
              secondary_y=False, 
              row=3, col=1)

# fig에 go.Scatter를 사용해서 ROAS 라인+마커 그래프 그리기
fig.add_trace(
    go.Scatter(
        x = [allRoasDf['create_year_month'].tolist(), allRoasDf['ltv_period'].tolist()], 
        y = allRoasDf['roas'].tolist(), 
        name='ROAS(all)', 
        mode = 'lines+markers+text', 
        text = allRoasDf['roas'], 
        textposition = 'top center',
        texttemplate = "%{text:.0%}", # %{변수:변수포맷}’의 형태로 사용.  https://github.com/d3/d3-format/tree/v1.4.5#d3-format
    ),
    secondary_y = True,
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x = [organicRoasDf['create_year_month'].tolist(), organicRoasDf['ltv_period'].tolist()], 
        y = organicRoasDf['roas'].tolist(), 
        name='ROAS(organic)', 
        mode = 'lines+markers+text', 
        text = organicRoasDf['roas'], 
        textposition = 'top center',
        texttemplate = "%{text:.0%}",
    ),
    secondary_y = True,
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x = [nonOrganicRoasDf['create_year_month'].tolist(), nonOrganicRoasDf['ltv_period'].tolist()], 
        y = nonOrganicRoasDf['roas'].tolist(), 
        name='ROAS(non-organic)', 
        mode = 'lines+markers+text', 
        text = nonOrganicRoasDf['roas'], 
        textposition = 'top center',
        texttemplate = "%{text:.0%}",
    ),
    secondary_y = True,
    row=3, col=1
)

# 출력되는 그래프 레이아웃 상세 조정
fig.update_layout(height = 900) # e.g.(height=600, width=600)
fig.update_layout(
    {
        "title": {
            "text": "<b>< 월간 가입유저 LTV, ROAS ><b>",
            "x": 0.5, # An int or float in the interval [0, 1]
            "y": 0.95, # An int or float in the interval [0, 1]
            "font": {
                "size": 13
            }
        },
        "yaxis1": {
            "title": "LTV (원)"
        },
        "yaxis2": { 
            "title": "ROAS",
            "tickformat": '.0%',
            "range": [0, 3],
        },
        "yaxis3": {
            "title": "LTV (원)"
        },
        "yaxis4": { 
            "title": "ROAS",
            "tickformat": '.0%',
            "range": [0, 3],
        },
        "yaxis5": {
            "title": "LTV (원)"
        },
        "yaxis6": { 
            "title": "ROAS",
            "tickformat": '.0%',
            "range": [0, 3],
        },
        
    },    
)

# 각 서브플랏에서, 이중축으로 잡혀있는 ROAS 그리드 지우기 
fig['layout']['yaxis2']['showgrid'] = False
fig['layout']['yaxis4']['showgrid'] = False
fig['layout']['yaxis6']['showgrid'] = False

# 그래프 출력
fig.show(renderer='colab')

In [ ]:
roasDf.head(20).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ltv_period         20 non-null     object 
 1   create_year_month  20 non-null     object 
 2   attribute_type     20 non-null     object 
 3   user_cnt           20 non-null     Int64  
 4   sales              20 non-null     Int64  
 5   ltv                20 non-null     float64
 6   cost               20 non-null     float64
 7   roas               20 non-null     float64
dtypes: Int64(2), float64(3), object(3)
memory usage: 1.4+ KB


### 캠페인별 월간 모객유저 ROAS

In [ ]:
## << 데이터프레임 생성 - 캠페인별 월간 가입 유저수  >>

query_job = client.query("""
  with userinfo as (
    select b.attribute_type, b.campaign
        , b.install_time, a.create_date, a.create_year_month
        , a.attribute_id, a.user_id, a.nickname, a.account_type, a.platform
        , a.region_1depth_name, a.region_2depth_name, a.region_3depth_name
        , a.h_code, a.x, a.y
    from `my_temp.marketing_myinfos` as a
    left join (
      select 'organic' as attribute_type, install_time, attribute_id, campaign
      from `my_temp.organic_install`

      union all 

      select 'non_organic' as attribute_type, install_time, attribute_id, campaign
      from `my_temp.non_organic_install`
    ) as b
    on a.attribute_id = b.attribute_id and a.create_date = b.install_time
  ),

  daily_orders as (
    select order_date, user_id, sum(payment_amount) as payment_amount
    from `my_temp.daily_orders`
    where order_date >= '2022-05-01'
    group by order_date, user_id
  ),

  ltv_set as (
    select 'D15' as ltv_period, a.user_id, a.campaign   
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 14 then 0 else d15.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d15
    on a.user_id = d15.user_id and a.create_date <= d15.order_date and date_add(a.create_date, interval 14 day) >= d15.order_date 

    union all

    select 'D30' as ltv_period, a.user_id, a.campaign
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 29 then 0 else d30.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d30
    on a.user_id = d30.user_id and a.create_date <= d30.order_date and date_add(a.create_date, interval 29 day) >= d30.order_date 

    union all

    select 'D45' as ltv_period, a.user_id, a.campaign
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 44 then 0 else d45.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d45
    on a.user_id = d45.user_id and a.create_date <= d45.order_date and date_add(a.create_date, interval 44 day) >= d45.order_date 

    union all

    select 'D60' as ltv_period, a.user_id, a.campaign
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 59 then 0 else d60.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d60
    on a.user_id = d60.user_id and a.create_date <= d60.order_date and date_add(a.create_date, interval 59 day) >= d60.order_date 

    union all

    select 'D75' as ltv_period, a.user_id, a.campaign
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 74 then 0 else d75.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d75
    on a.user_id = d75.user_id and a.create_date <= d75.order_date and date_add(a.create_date, interval 74 day) >= d75.order_date 

    union all

    select 'D90' as ltv_period, a.user_id, a.campaign
        , a.create_year_month, a.attribute_type
        , case when date_diff(DATE '2022-11-30', a.create_date, DAY) < 89 then 0 else d90.payment_amount end as payment_amount
    from userinfo as a
    left join daily_orders as d90
    on a.user_id = d90.user_id and a.create_date <= d90.order_date and date_add(a.create_date, interval 89 day) >= d90.order_date 

  ),

  agg_ltv_group as (
    select ltv_period, create_year_month, attribute_type, campaign
        , count(distinct user_id) as user_cnt
        , sum(payment_amount) as sales
        , round(sum(payment_amount)/count(distinct user_id)) as ltv 
        , count(distinct case when payment_amount = 0 then user_id end) as check_cnt
    from ltv_set
    group by rollup(ltv_period, create_year_month, attribute_type, campaign)
  ),

  agg_ltv_group_final as (
    select ltv_period, create_year_month
        , case when attribute_type is null then 'all' else attribute_type end as attribute_type
        , campaign
        , user_cnt
        , case when check_cnt = 0 then sales else 0 end as sales
        , case when check_cnt = 0 then ltv else 0 end as ltv
    from agg_ltv_group
    where create_year_month is not null and campaign is not null
      and attribute_type = 'non_organic'
    order by ltv_period, create_year_month, attribute_type
  ), 

  monthly_cost as (
    select format_date('%Y-%m', aggregate_date) as aggregate_year_month, campaign, sum(cost) as cost
    from `my_temp.daily_cost`
    group by aggregate_year_month, campaign
    order by aggregate_year_month, campaign
  )

  select a.ltv_period, a.create_year_month, a.attribute_type, a.campaign
      , a.user_cnt, a.sales, a.ltv, b.cost
      , round(a.sales/b.cost, 3) as roas 
  from agg_ltv_group_final as a
  left join monthly_cost as b
  on a.create_year_month = b.aggregate_year_month and a.campaign = b.campaign
  order by create_year_month, attribute_type, ltv_period

""")

roasByCampaignDf = query_job.to_dataframe()

In [ ]:
# 11월은 지표 추적기간이 다차지 않았으므로 데이터에서 제외 처리
seoulRoasDf = roasByCampaignDf[(roasByCampaignDf['campaign'] == 'SEOUL') & (roasByCampaignDf['create_year_month'] != '2022-11')] 
mokdongRoasDf = roasByCampaignDf[(roasByCampaignDf['campaign'] == 'LC_Mok-dong') & (roasByCampaignDf['create_year_month'] != '2022-11')]
gurodongDf = roasByCampaignDf[(roasByCampaignDf['campaign'] == 'LC_Guro-dong') & (roasByCampaignDf['create_year_month'] != '2022-11')]
mullaedongRoasDf = roasByCampaignDf[(roasByCampaignDf['campaign'] == 'LC_Mullae-dong') & (roasByCampaignDf['create_year_month'] != '2022-11')]
sillimdongRoasDf = roasByCampaignDf[(roasByCampaignDf['campaign'] == 'LC_Sillim-dong') & (roasByCampaignDf['create_year_month'] != '2022-11')]
dangsandongRoasDf = roasByCampaignDf[(roasByCampaignDf['campaign'] == 'LC_Dangsan-dong') & (roasByCampaignDf['create_year_month'] != '2022-11')]
yeouidodongRoasDf = roasByCampaignDf[(roasByCampaignDf['campaign'] == 'LC_Yeouido-dong') & (roasByCampaignDf['create_year_month'] != '2022-11')]
sindorimdongRoasDf = roasByCampaignDf[(roasByCampaignDf['campaign'] == 'LC_Sindorim-dong') & (roasByCampaignDf['create_year_month'] != '2022-11')]


import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 그리고자 하는 서브플랏 골격 설정 (2x1로 설정, 그래프간 간격 설정)
fig = make_subplots(rows=2, cols=1, 
                    vertical_spacing = 0.15)

# fig에 go.Bar를 사용해서 캠페인별 가입 유저수 막대그래프 그리기 
fig.add_trace(go.Bar(x = [seoulRoasDf['create_year_month'].tolist(), seoulRoasDf['ltv_period'].tolist()], 
                     y= seoulRoasDf['user_cnt'].tolist(), name = "유저수(SEOUL)"), 
              row=1, col=1)  

fig.add_trace(go.Bar(x = [mokdongRoasDf['create_year_month'].tolist(), mokdongRoasDf['ltv_period'].tolist()], 
                     y= mokdongRoasDf['user_cnt'].tolist(), name = "유저수(mokdong)"), 
              row=1, col=1)  

fig.add_trace(go.Bar(x = [gurodongDf['create_year_month'].tolist(), gurodongDf['ltv_period'].tolist()], 
                     y= gurodongDf['user_cnt'].tolist(), name = "유저수(gurodong)"), 
              row=1, col=1)  

fig.add_trace(go.Bar(x = [mullaedongRoasDf['create_year_month'].tolist(), mullaedongRoasDf['ltv_period'].tolist()], 
                     y= mullaedongRoasDf['user_cnt'].tolist(), name = "유저수(mullaedong)"), 
              row=1, col=1)

fig.add_trace(go.Bar(x = [sillimdongRoasDf['create_year_month'].tolist(), sillimdongRoasDf['ltv_period'].tolist()], 
                     y= sillimdongRoasDf['user_cnt'].tolist(), name = "유저수(sillimdong)"), 
              row=1, col=1)  

fig.add_trace(go.Bar(x = [dangsandongRoasDf['create_year_month'].tolist(), dangsandongRoasDf['ltv_period'].tolist()], 
                     y= dangsandongRoasDf['user_cnt'].tolist(), name = "유저수(dangsandong)"), 
              row=1, col=1)  

fig.add_trace(go.Bar(x = [yeouidodongRoasDf['create_year_month'].tolist(), yeouidodongRoasDf['ltv_period'].tolist()], 
                     y= yeouidodongRoasDf['user_cnt'].tolist(), name = "유저수(yeouidodong)"), 
              row=1, col=1)  

fig.add_trace(go.Bar(x = [sindorimdongRoasDf['create_year_month'].tolist(), sindorimdongRoasDf['ltv_period'].tolist()], 
                     y= sindorimdongRoasDf['user_cnt'].tolist(), name = "유저수(sindorimdong)"), 
              row=1, col=1)  


# fig에 go.Bar를 사용해서 캠페인별 ROAS 막대그래프 그리기 
fig.add_trace(go.Bar(x = [seoulRoasDf['create_year_month'].tolist(), seoulRoasDf['ltv_period'].tolist()], 
                     y= seoulRoasDf['roas'].tolist(), name = "ROAS(SEOUL)"), 
              row=2, col=1)  

fig.add_trace(go.Bar(x = [mokdongRoasDf['create_year_month'].tolist(), mokdongRoasDf['ltv_period'].tolist()], 
                     y= mokdongRoasDf['roas'].tolist(), name = "ROAS(mokdong)"), 
              row=2, col=1)  

fig.add_trace(go.Bar(x = [gurodongDf['create_year_month'].tolist(), gurodongDf['ltv_period'].tolist()], 
                     y= gurodongDf['roas'].tolist(), name = "ROAS(gurodong)"), 
              row=2, col=1)  

fig.add_trace(go.Bar(x = [mullaedongRoasDf['create_year_month'].tolist(), mullaedongRoasDf['ltv_period'].tolist()], 
                     y= mullaedongRoasDf['roas'].tolist(), name = "ROAS(mullaedong)"), 
              row=2, col=1)

fig.add_trace(go.Bar(x = [sillimdongRoasDf['create_year_month'].tolist(), sillimdongRoasDf['ltv_period'].tolist()], 
                     y= sillimdongRoasDf['roas'].tolist(), name = "ROAS(sillimdong)"), 
              row=2, col=1)  

fig.add_trace(go.Bar(x = [dangsandongRoasDf['create_year_month'].tolist(), dangsandongRoasDf['ltv_period'].tolist()], 
                     y= dangsandongRoasDf['roas'].tolist(), name = "ROAS(dangsandong)"), 
              row=2, col=1)  

fig.add_trace(go.Bar(x = [yeouidodongRoasDf['create_year_month'].tolist(), yeouidodongRoasDf['ltv_period'].tolist()], 
                     y= yeouidodongRoasDf['roas'].tolist(), name = "ROAS(yeouidodong)"), 
              row=2, col=1)  

fig.add_trace(go.Bar(x = [sindorimdongRoasDf['create_year_month'].tolist(), sindorimdongRoasDf['ltv_period'].tolist()], 
                     y= sindorimdongRoasDf['roas'].tolist(), name = "ROAS(sindorimdong)"), 
              row=2, col=1)  


# 출력되는 그래프 레이아웃 상세 조정
fig.update_layout(height = 900) # e.g.(height=600, width=600)
fig.update_layout(
    {
        "title": {
            "text": "<b>< 월간 가입유저 LTV, ROAS ><b>",
            "x": 0.5, # An int or float in the interval [0, 1]
            "y": 0.95, # An int or float in the interval [0, 1]
            "font": {
                "size": 13
            }
        },
        "yaxis": { 
            "title": "가입유저수(명)",
        },
        "yaxis2": { 
            "title": "ROAS",
            "tickformat": '.0%',
            "range": [0, 3],
        },
    },    
)

# 그래프 출력
fig.show(renderer='colab')